In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

 
###############################################################################
#                       Inputting and initializing                            #
###############################################################################


# Inputting the file name
filename = input("Enter a file name: ")

# Inputting the numbers of comparison stars
num_comp_stars = input("Enter the number of comparison stars: ")
num_comp_stars = int(num_comp_stars)

# Inputting start and end times in JD UTC
start_time = input("Enter the transit start time in JD UTC: ")
start_time = float(start_time)
end_time = input("Enter the transit end time in JD UTC: ")
end_time = float(end_time)

# Loading in the data using pandas
df = pd.read_csv(filename)

# Initializing the time lists
JD_2400000 = list(df['J.D.-2400000'])
JD_UTC = list(df['JD_UTC'])

# Initializing the target star information
source_sky_T1 = list(df['Source-Sky_T1'])
source_sky_T1_err = list(df['Source_Error_T1'])

# Filling the comparison star lists (list of lists)
source_sky_list = [] # Will contain all of the comparison star source-sky values
source_sky_err_list = [] # Errors for comparison star source-sky values

# Filling above comparison star lists with lists
for comp_star in range(2,num_comp_stars+2):
    #generates a list of lists to hold all source-sky data
    comp_star_col = 'Source-Sky_C' + str(comp_star)
    comp_star_err_col = 'Source_Error_C' + str(comp_star)
    source_sky_list.append(list(df[comp_star_col]))
    source_sky_err_list.append(list(df[comp_star_err_col]))
               

Enter a file name: Measurements.csb
Enter the number of comparison stars: 5
Enter the transit start time in JD UTC: 4
Enter the transit end time in JD UTC: 4


FileNotFoundError: [Errno 2] File b'Measurements.csb' does not exist: b'Measurements.csb'

In [2]:
###############################################################################
#                     Magnitude and Error Calculations                        #
###############################################################################


def mag_calc(source_sky_T1, source_sky_C):
    ''' Takes target star and comparison star lists as inputs. Loops through
    them and calculates their relative magnitude, returned as a list.'''
    
    mag = []
    for i in range(len(source_sky_T1)):
        mag.append(((-2.5) * np.log10(source_sky_T1[i]/source_sky_C[i])))
        
    return mag

def err_calc(source_sky_T1, source_sky_T1_err, source_sky_C, source_sky_C_err):
    ''' Takes target star and comparison star value and error lists as inputs. 
    Loops through them and calculates their error returned as a list.'''
    
    err = []
    
    for i in range(len(source_sky_T1)):
        part1 = (-2.5 / np.log(10)) * (1 / source_sky_T1[i]) * source_sky_T1_err[i]
        part2 = (-2.5 / np.log(10)) * (1 / source_sky_C[i]) * source_sky_C_err[i]
        err.append(np.sqrt((part1**2) + (part2**2)))
        
    return err

In [3]:
###############################################################################
#                                 Plotting                                    #
###############################################################################


def plotting(source_sky_T1, source_sky_T1_err, source_sky_C, source_sky_C_err):
    ''' Takes target star and comparison star value and error lists as inputs. 
    Performs magnitude and error calculations on them then plots the data.'''
    
    rel_mag_T1C = mag_calc(source_sky_T1, source_sky_C)
    
    err = err_calc(source_sky_T1, source_sky_T1_err, source_sky_C, source_sky_C_err)
    
    plt.errorbar(JD_UTC,rel_mag_T1C, yerr = err, marker='d', markersize =5, linestyle='')
        
    plt.gca().invert_yaxis()
        
    plt.xlabel("JD UTC")
    plt.ylabel("Relative Magnitude")
    plt.axvline(start_time, color = 'r') # Plots vertical line at inputted start time
    plt.axvline(end_time, color = 'r') # Plots vertical line at inputted end time
    
    
def plot_all_mags():
    '''Loops through all comparison stars and performs plotting function on all
    of them. Makes text more legible and saves figures.'''
    
    for x in range(len(source_sky_list)):
        plotting(source_sky_T1, source_sky_T1_err, source_sky_list[x], source_sky_err_list[x])
        plt.title("Relative Brightness of\nTarget Star to Comp Star " + str(x + 2))

        SMALL_SIZE = 12
        MEDIUM_SIZE = 15
        BIGGER_SIZE = 20

        plt.rc('font', size=BIGGER_SIZE)         # controls default text sizes
        plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
        plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        plt.tight_layout()
        plt.savefig("MagnitudeT1toC" + str(x + 2))
        #plt.savefig("PosterPlots/MagnitudeT1toC" + str(x + 2))
        plt.show()

In [ ]:
###############################################################################
#                          Running Everything                                 #
###############################################################################

# Running function to create and save all plots   
plot_all_mags()           